In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
# from sklearn import qda, lda
import math
from numpy.linalg import inv
import lda
import qda
import lr

In [ ]:
url = 'https://raw.githubusercontent.com/probml/pmtk1/master/pmtk/data/heightWeightData.txt'
data = pd.read_csv(url, header = None)

In [ ]:
X = data.ix[:, 1 : 2].values
y = data.ix[:, 0].values
y = (y == 1) * y

In [ ]:
clf = lr.LogisticRegression()
clf.fit(X, y)
np.mean(clf.predict(X) == y)

In [ ]:
X = np.column_stack((X, X[:, 0] ** 2))
X = np.column_stack((X, X[:, 1] ** 2))
X = np.column_stack((X, X[:, 0] * X[:, 1]))

In [ ]:
clf = lr.LogisticRegression()
clf.fit(X, y)
np.mean(clf.predict(X) == y)

In [ ]:
clf = qda.QDA()
clf.fit(X, y)
np.mean(clf.predict(X) == y)

In [ ]:
X = (X - np.mean(X, axis = 0)) / np.std(X, axis = 0)
clf = lda.LDA()
clf.fit(X, y)
np.mean(clf.predict(X) == y)